# Домашнее задание к лекции «Продвинутый pandas»

<a id='contents'></a>
## Оглавление:
### [Задание 1](#task1)
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

Результат классификации запишите в столбец class
### [Задание 2](#task2)
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном
### [Задание 3](#task3)
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.
### [Задание 4](#task4)
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

<a id='task1'></a>
## Задание 1

In [2]:
import pandas as pd

In [120]:
route = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\NUMPY-21_practical_works\2021.06.28_Продвинутый Pandas\12_доп_материалы_к_пандас_1_и_2\visit_log.csv'

df = pd.read_csv(route, sep = ';') # читаем исходник csv в dataframe, прямо указывая разделитель
df.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [92]:
# создаем словарь для проверки
# в качестве ключей задаем проверяемые параметры переводя их в неизменяемый
# список (tuple), иначе словарь список в ключ не примет
check_dic = {tuple(['paid', 'email', 'Russia']):'ad',
             tuple(['yandex', 'google', 'any country']):'organic',
             tuple(['paid', 'email', 'not Russia']):'other'
}

In [95]:
def source (row):
    '''
    Во-первых проверяется, относится ли источник к 'paid', 'email' из 'Russia'.
    Если не выполняется, делается проверка, что источник есть среди ключей словаря, но в текущий момент
    цикл не стоит на элементе словаря с 'Russia', чтобы не допустить ошибку, когда источник из 'paid', 'email' не в России
    '''
    for key, value in check_dic.items():
        if ((row.traffic_source in key) and               # проверяем, что в строке есть 'paid', 'email' и 'Russia'
            (row.region in key)):
            return value
        elif ((row.traffic_source in key) and             # проверяем, что в строке есть 'paid', 'email', 'yandex', 'google', но
                                                          # но мы не стоим на строке словаря, содержащим 'Russia'
              (list(check_dic.keys())[0][2] not in key)):
            return value
    return row.traffic_source                             # если ничего не подходит - выводим название источника

Тестируем функцию на тестовом DataFrame

In [96]:
test_df = pd.DataFrame({'timestamp':['NaN'],
                        'visit_id':['NaN'],
                        'url':['NaN'],
                        'region':['Russia'],
                        'user_id':['NaN'],
                        'traffic_source':['email']})
test_df['source_type'] = test_df.apply(source, axis = 1)
test_df.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,NaN,NaN,NaN,Russia,NaN,email,ad


In [94]:
df['source_type'] = df.apply(source, axis = 1)
df.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [78]:
%%time
for i in range(10):
    df['source_type'] = df.apply(source, axis = 1)

Wall time: 6.95 s


### Альтернативный вариант (более быстрый)

In [118]:
df.loc[(df.traffic_source.isin(['paid', 'email'])) & (df.region == 'Russia'), 'source_type', ] = 'ad'
df.loc[(df.traffic_source.isin(['paid', 'email'])) & (df.region != 'Russia'), 'source_type'] = 'other'
df.loc[df.traffic_source.isin(['yandex', 'google']), 'source_type'] = 'organic'
df.source_type.fillna(df.traffic_source, inplace = True)
df.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [126]:
df.groupby(['source_type', 'traffic_source', 'region']).count()

timestamp  visit_id   url  user_id
source_type traffic_source region                                       
ad          email          Russia          2209      2209  2209     2209
            paid           Russia          1581      1581  1581     1581
direct      direct         Australia        222       222   222      222
                           Belarus          998       998   998      998
                           China            397       397   397      397
                           India            134       134   134      134
                           Russia          4052      4052  4052     4052
                           Ukraine         1210      1210  1210     1210
organic     google         Australia         91        91    91       91
                           Belarus          376       376   376      376
                           China            151       151   151      151
                           India             55        55    55       55
                           Russia          1435      1435  1435     1435
                           Ukraine          399       399   399      399
            yandex         Australia         88        88    88       88
                           Belarus          446       446   446      446
                           China            171       171   171      171
                           India             59        59    59       59
                           Russia          1636      1636  1636     1636
                           Ukraine          500       500   500      500
other       email          Australia        115       115   115      115
                           Belarus          569       569   569      569
                           China            252       252   252      252
                           India             82        82    82       82
                           Ukraine          648       648   648      648
            paid           Australia         74        74    74       74
                           Belarus          368       368   368      368
                           China            138       138   138      138
                           India             39        39    39       39
                           Ukraine          443       443   443      443

In [121]:
%%time
for i in range(10):
    df.loc[(df.traffic_source.isin(['paid', 'email'])) & (df.region == 'Russia'), 'source_type'] = 'ad'
    df.loc[(df.traffic_source.isin(['paid', 'email'])) & (df.region != 'Russia'), 'source_type'] = 'other'
    df.loc[df.traffic_source.isin(['yandex', 'google']), 'source_type'] = 'organic'
    df.source_type.fillna(df.traffic_source, inplace = True)
df.head()

Wall time: 78 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


[к оглавлению](#contents)

<a id='task2'></a>
## Задание 2

In [130]:
route = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\NUMPY-21_practical_works\2021.06.28_Продвинутый Pandas\12_доп_материалы_к_пандас_1_и_2\URLs.txt'

df = pd.read_csv(route) # читаем исходник txt в dataframe
df.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [132]:
pattern = r'\/\d{8}-'
df.loc[df.url.str.contains(pattern, regex=True)]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


[к оглавлению](#contents)

<a id='task3'></a>
## Задание 3

In [195]:
route = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\NUMPY-21_practical_works\2021.06.28_Продвинутый Pandas\12_доп_материалы_к_пандас_1_и_2\ml-latest-small\ratings.csv'

df = pd.read_csv(route) # читаем исходник csv в dataframe
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [196]:
import numpy as np

# производим группировку с одновременной агрегацией: считаем количество записей по рейтингам в рамках одного ID, а также
# минимальное и максимальное значение timestamp для каждого id
df_grouped = df.groupby('userId').agg({'rating': 'count', 'timestamp': [np.min, np.max]})
df_grouped.head()

rating   timestamp            
        count        amin        amax
userId                               
1          20  1260759108  1260759205
2          76   835355395   835356246
3          51  1298861589  1298932787
4         204   949778714   949982274
5         100  1163373044  1163375145

In [198]:
# отфильтровываем, оставляя только id с количеством оценок более 100, одновременно создавая для них новый столбец life_time,
# представляющий собой разницу между максимальным и минимальным временем входа (timestamp)
df_grouped.loc[df_grouped.rating['count'] > 100, 'life_time'] = df_grouped.timestamp['amax'] - df_grouped.timestamp['amin']
# исключаем всех, кто не соответствовал фильтру, т.е. timestamp = NaN
df_grouped = df_grouped.loc[~df_grouped.life_time.isnull()]
df_grouped.head()

rating   timestamp              life_time
        count        amin        amax           
userId                                          
4         204   949778714   949982274     203560
8         116  1154389340  1154474527      85187
15       1700   997937239  1469330735  471393496
17        363  1127468587  1127476640       8053
19        423   855190091   855195373       5282

In [200]:
df_grouped.life_time.mean() # считаем среднее по столбцуф

40080507.4496124

[к оглавлению](#contents)

<a id='task4'></a>
## Задание 4

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [202]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [203]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [204]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [205]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


Таблица с тремя типами выручки для каждого client_id без указания адреса клиента

In [217]:
merged_table = rzd.merge(auto, how='outer').merge(air, how='outer')
merged_table.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


Таблица по типам выручки с указанием адреса клиента

In [219]:
full_table = client_base.merge(merged_table,  how='left')
full_table.fillna(0)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


[к оглавлению](#contents)